In [87]:
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

In [88]:
print(azureml.core.VERSION)

1.38.0


In [89]:
AZ_SUBSCRIPTION_ID='4dd39263-9f5d-4297-abe2-3bfd4428480b'
ws = Workspace.create(name='new_predict',
                      subscription_id=AZ_SUBSCRIPTION_ID, 
                      resource_group='ml',
                      create_resource_group=True,
                      location='southeastasia'
                     )

Deploying StorageAccount with name newpredistorageac79e5379.
Deploying KeyVault with name newpredikeyvault8394cb8c.
Deploying AppInsights with name newprediinsightsc26dc20e.
Deployed AppInsights with name newprediinsightsc26dc20e. Took 4.08 seconds.
Deployed KeyVault with name newpredikeyvault8394cb8c. Took 17.9 seconds.
Deploying Workspace with name new_predict.
Deployed StorageAccount with name newpredistorageac79e5379. Took 30.27 seconds.
Deployed Workspace with name new_predict. Took 35.87 seconds.


In [90]:
ws.write_config()
ws.get_details()

{'id': '/subscriptions/4dd39263-9f5d-4297-abe2-3bfd4428480b/resourceGroups/ml/providers/Microsoft.MachineLearningServices/workspaces/new_predict',
 'name': 'new_predict',
 'identity': {'principal_id': 'e523d64d-0a9f-4791-842d-fc8b97b0b725',
  'tenant_id': '5ba5ef5e-3109-4e77-85bd-cfeb0d347e82',
  'type': 'SystemAssigned'},
 'location': 'southeastasia',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': 'd551c561-26d3-4424-b172-a02c7578fe7f',
 'sdkTelemetryAppInsightsKey': '0855780c-10d5-4461-ae3c-9b15eb18c90d',
 'description': '',
 'friendlyName': 'new_predict',
 'creationTime': '2022-02-16T06:48:04.7144136+00:00',
 'keyVault': '/subscriptions/4dd39263-9f5d-4297-abe2-3bfd4428480b/resourcegroups/ml/providers/microsoft.keyvault/vaults/newpredikeyvault8394cb8c',
 'applicationInsights': '/subscriptions/4dd39263-9f5d-4297-abe2-3bfd4428480b/resourcegroups/ml/providers/microsoft.insights/components/newprediinsightsc26dc20e',
 'storageAccount

In [91]:
exp = Experiment(workspace=ws, name='newxp')
run = exp.start_logging()                   
run.log("Experiment start time", str(datetime.datetime.now()))

In [92]:
train_old_project = pd.read_csv('land_transaction_data.csv',header=0, index_col=None)
x_train = train_old_project[["land_txn_price"]]
y_train = train_old_project["openning_price"]
linReg = LinearRegression()
linReg_model = linReg.fit(x_train, y_train)

In [93]:
filename = "predict_new_model.pkl"
joblib.dump(linReg_model, filename)

['predict_new_model.pkl']

In [94]:
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

In [81]:
print(run.get_portal_url())

https://ml.azure.com/runs/78eb2109-4d30-4ec4-bb97-d1bb2bf4125f?wsid=/subscriptions/4dd39263-9f5d-4297-abe2-3bfd4428480b/resourcegroups/ml/workspaces/new_predict&tid=5ba5ef5e-3109-4e77-85bd-cfeb0d347e82


In [95]:
model = Model.register(model_path = "predict_new_model.pkl",
                       model_name = "predict_new_model",
                       tags = {"key": "1"},
                       description = "Newly launched project price prediction",
                       workspace = ws)

Registering model predict_new_model


In [96]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Price",  "method" : "sklearn"}, 
                                               description='Predict new project price')

In [83]:
salenv = CondaDependencies()
salenv.add_conda_package("scikit-learn")

with open("salenv.yml","w") as f:
    f.write(salenv.serialize_to_string())
with open("salenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



In [97]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('predict_new_model')
    model = joblib.load(model_path)

def run(raw_data):
    data = json.loads(raw_data)['num']
    y_hat = model.predict(data)
    return json.dumps(y_hat)

Overwriting score.py


In [98]:
%%time
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="salenv.yml")

Wall time: 995 µs


<timed exec>:1: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments


In [99]:
service = Webservice.deploy_from_model(workspace=ws,
                                       name='predict-price-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

<ipython-input-99-329dc489caab>:1: DeprecationWarning: deploy_from_model has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  service = Webservice.deploy_from_model(workspace=ws,


Creating image


Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Wed, 16 Feb 2022 06:56:55 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'x-ms-client-request-id': '04346231-879f-49d1-b952-9802c7fef430', 'x-ms-client-session-id': '2610ba9f-5739-4573-b64b-7145928094f0', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.045', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"OperationNotFound","message":"There is no operation with id 1eb621bb-5651-4e8b-a26e-f0ad29e22949"}],"correlation":{"RequestId":"04346231-879f-49d1-b952-9802c7fef430"}}'



WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Wed, 16 Feb 2022 06:56:55 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'x-ms-client-request-id': '04346231-879f-49d1-b952-9802c7fef430', 'x-ms-client-session-id': '2610ba9f-5739-4573-b64b-7145928094f0', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.045', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"OperationNotFound","message":"There is no operation with id 1eb621bb-5651-4e8b-a26e-f0ad29e22949"}],"correlation":{"RequestId":"04346231-879f-49d1-b952-9802c7fef430"}}'
	InnerException None
	ErrorResponse 
{
    "error"

In [63]:
!curl -X POST \
	-H 'Content-Type':'application/json' \
	-d '''{"num":600}''' \
	http://540e88e2-afeb-406c-b487-696b0cc1049d.southeastasia.azurecontainer.io/score

{"message": "Expects Content-Type to be application/json"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    73  100    58  100    15    331     85 --:--:-- --:--:-- --:--:--   421


In [62]:
print(service.scoring_uri)

http://540e88e2-afeb-406c-b487-696b0cc1049d.southeastasia.azurecontainer.io/score


In [ ]:
dd = '''{"num":600}'''
def run(raw_data):
    data = json.loads(raw_data)['num']
    print(type(data))
    return json.dumps(data)

run(dd)